# Matrices

In [2]:
import numpy as np

def connectivity(data):
    '''Compute Pearson correlation for every patient'''
    ## data - nTime x nChannels x nPatients
    if data.ndim == 1:
        connMatrix = -1
    elif data.ndim == 2:
        connMatrix = np.corrcoef(data.T)
    else:
        connMatrix = np.zeros((np.size(data,2),np.size(data,1),np.size(data,1)))
        for i in range(np.size(data,2)):
            dataTmp = data[:,:,i]
            connMatrix[i] = np.corrcoef(dataTmp.T)
    return connMatrix

In [4]:
# def orderMatrix(connMatrix):
#     #values sorted max to min, as usual in correlation business; 
#     var = -connMatrix 
#     corr_mat = np.zeros(np.shape(var))
#     sorted_num = np.sort(np.unique(var[var!=0]))
#     for count in range(0,len(sorted_num)):
#         var[np.where(var==sorted_num[count])]=count+1
#         corr_mat[:,:]=var
#     return corr_mat

import numpy as np
def orderMatrix_single(connMatrix):
    '''Turn matrix it its ordered version. Diagonal 0'''
    var = connMatrix # values sorted min to max. Be carful, for correlation sort max to min (BY HAND), as usual in correlation business; 
    N = np.size(var,0)
    values = var[np.triu_indices(N,1)] # symmetric matrix to vector of upper triangle values
    order = vectorOrder(values)+1 # from values to order
    upMatr = createTriMatr_upper(order,N,1) # back to upper triangular matrix (diagonal 0)
    ordMatrix = upMatr + upMatr.T # create symmetric matrix
    return ordMatrix


def orderMatrix(connMatrix):
    '''Find order matrix for every patient'''
    ## connMatrix - nChannels x nChannels x nPatients
    ordMatrix = np.zeros(np.shape(connMatrix))
    if ordMatrix.ndim == 2:
        ordMatrix = orderMatrix_single(connMatrix)
    else: 
        for i in range(np.size(connMatrix,0)):
            ordMatrix[i] = orderMatrix_single(connMatrix[i])
    return ordMatrix

In [12]:
import numpy as np
def shuffleMatrix_single(N):
    '''Create symmetric random matrix NxN with uniform distribution'''
    rand_vect = (np.random.permutation(binom(N)))
    rand_mat = createTriMatr_upper(rand_vect,N,1)
    rand_mat = rand_mat + rand_mat.T
    return rand_mat


def shuffleMatrix(N,K=0):
    '''Radnom matrix for every patient'''
    ## output matrix - N (Channels) x N (Channels) x K (Patients)
    if K == 0:
        ordMatrix = shuffleMatrix_single(N)
    else: 
        ordMatrix = np.zeros((K,N,N))
        for i in range(K):
            ordMatrix[i] = shuffleMatrix_single(N)
    return ordMatrix

In [13]:
import numpy as np

def randCorrMatrix(N, K = 1, Len = 400):
    '''Radnom correlation matrix for every patient'''
    ## output matrix - K (Patients) x N (Channels) x N (Channels)
    ts = np.random.normal(size=(Len,N,K))
    randCorrMatrix = connectivity(ts)
    return randCorrMatrix

In [1]:
import numpy as np
from scipy.spatial import distance

def euclDistMatrix_single(N_sample, dim_space = 400):
    geom_mat = np.zeros((N_sample,N_sample))
    point_sample = np.random.uniform(low = 0, high = 1, size = (N_sample,dim_space))
    for s in range(0,N_sample):
        for t in range(s+1,N_sample):
            geom_mat[s,t] = distance.euclidean(point_sample[s],point_sample[t])
            geom_mat[t,s] = geom_mat[s,t]
    # var = geom_mat
    # sorted_num=np.sort(np.unique(var[var>0]))
    # for count in range(0,binom(N_sample)):
    #     var[np.where(var==sorted_num[count])]=count+1
    #     geom_mat = var
    return geom_mat
    
def euclDistMatrix(N, K = 0, dim_space = 400):
    '''euclidean distance matrix for every patient'''
    ## output matrix - K (Patients) x N (Channels) x N (Channels)
    if K == 0:
        outMatrix = euclDistMatrix_single(N,dim_space)
    else: 
        outMatrix = np.zeros((K,N,N))
        for i in range(K):
            outMatrix[i] = euclDistMatrix_single(N,dim_space)
    return outMatrix

In [15]:
import math
import numpy as np

def hyperToEucl(hypPoints):
    '''Function from [0,R] in hyperbolic geometry to [0,L<1] in euclidean'''
    eucl = (np.cosh(hypPoints) - 1)/(2 + np.cosh(hypPoints))
    return eucl
    
def hyperDensity(r, R = 10):
    '''Compute hyperbolic density for radius r (max R)'''
    rho = (np.exp(r)-np.exp(-r))/(np.exp(R)-np.exp(-R))
    return rho

def sampleHyperDistr(N_sample = 1000, R = 10):
    '''Generate uniform distribution on [0,R] with hyp.distance'''
    u = np.random.uniform(0,R,np.int(np.ceil(N_sample*(R+2)))) # generate uniformly more points than we need
    rho = hyperDensity(u,R)  # compute hyperbolic density for them
    remove_probability = np.random.uniform(0,hyperToEucl(R),size = np.size(u))  # probability, used to remove points
    selPoints = u[remove_probability<rho]   # leave only rho percent of points
    indices = np.random.randint(1,np.size(selPoints),N_sample) # select exact number of points
    selPoints = selPoints[indices]
    return selPoints

    
def distribution_on_disk(type = 'UniOnHyper', N_sample = 400, R = 10, expScale = 1):
  if (type == 'Circle'):
    print(type)
    u = np.random.uniform(0.9999,1,N_sample)
  elif (type == 'Disk'):
    print(type)
    u = np.random.uniform(0,1,N_sample)
  elif (type == 'Expo'):
    print(type)
    u = np.random.exponential(scale = expScale, size = (1,N_sample))
    u[u>=1] = 0.99999999
    u = 1-u
    # u = u*np.exp(-R)
    print(np.max(u))
  elif 'UniOnHyper':
    hypPoints = sampleHyperDistr(N_sample,R)
    euclPoints = hyperToEucl(hypPoints)
    # euclPoints = hypPoints
    u = euclPoints
  else:
    u = np.random.uniform(0,1,N_sample)
  return u


def distance_hyper(x, n_sample):
    " X should be normalized (norm <1) "
    geom_mat = np.zeros((n_sample, n_sample))
    for s in range(0, n_sample):
        for t in range(s + 1, n_sample):
            norm_diff = np.power(np.linalg.norm(x[s] - x[t]), 2)
            norm_s = np.power(np.linalg.norm(x[s]), 2)
            norm_t = np.power(np.linalg.norm(x[t]), 2)
            delta_st = (2 * norm_diff) / ((1 - norm_s) * (1 - norm_t))
            geom_mat[s, t] = math.acosh(1 + delta_st)
            geom_mat[t, s] = geom_mat[s, t]
    return geom_mat


def hyperbDistMatrix_single(N_sample, dim_space=400, distrType = 'UniOnHyper', R = 10):

    vec = np.random.normal(size = (dim_space,N_sample))
    vec /= np.linalg.norm(vec, axis=0) #creates random points on the unit sphere

    u = distribution_on_disk(distrType, N_sample, R)
    # r = np.power(u,1/dim_space) #random radius
    
    X = u*vec #resulting random vectors
    X = X.transpose()

    geom_mat = distance_hyper(X,N_sample)
    return geom_mat

def hyperbDistMatrix(N, K = 0, dim_space = 400, distrType = 'UniOnHyper', R = 1):
    '''hyperbolic distance matrix for every patient'''
    ## output matrix - K (Patients) x N (Channels) x N (Channels)
    if K == 0:
        outMatrix = hyperbDistMatrix_single(N, dim_space, distrType, R)
    else: 
        outMatrix = np.zeros((K,N,N))
        for i in range(K):
            outMatrix[i] = hyperbDistMatrix_single(N, dim_space, distrType, R)
    return outMatrix

In [11]:
import math
import numpy as np

def spherDistMatrix_single(N_sample, dim_space=400):
    vec = np.random.normal(size=(dim_space,N_sample))
    vec /= np.linalg.norm(vec, axis=0) #creates random points on the unit sphere
    vec = vec.transpose()
    geom_mat = np.zeros((N_sample,N_sample))

    for s in range(0,N_sample):
        for t in range(s+1,N_sample):
            geom_mat[s,t] = np.arccos(np.dot(vec[s],vec[t]))
            geom_mat[t,s] = geom_mat[s,t]
    # var = geom_mat
    # sorted_num = np.sort(np.unique(var[var!=0]))
    # for count in range(0,binom(N_sample)):
    #     var[np.where(var==sorted_num[count])] = count+1
    #     geom_mat = var
    return geom_mat

def spherDistMatrix(N, K = 0, dim_space = 400):
    '''spherical distance matrix for every patient'''
    ## output matrix - K (Patients) x N (Channels) x N (Channels)
    if K == 0:
        outMatrix = spherDistMatrix_single(N,dim_space)
    else: 
        outMatrix = np.zeros((K,N,N))
        for i in range(K):
            outMatrix[i] = spherDistMatrix_single(N,dim_space)
    return outMatrix

# Persistent Homology 

In [9]:
import numpy as np
from ripser import ripser
from tqdm import tqdm

def computeBettiNumbers_single(ordMatr, minDim=0, maxDim=2):
    N_max = binom(np.size(ordMatr,0))
    N_trial = 1
    N_betti = {}
    for dim in range(0,maxDim+1):
        N_betti['Betti'+str(dim)+''] = {}
    diagrams = ripser(ordMatr, maxdim = maxDim, distance_matrix = True)['dgms']
    for dim in range(minDim,maxDim+1):
        N_betti['Betti'+str(dim)+''] = []
        betti = diagrams[dim]
        for index in range(0,N_max+1):
            wo = np.array(np.where((betti[:,0] <= index)&(betti[:,1]>=index)))
            N_betti['Betti'+str(dim)+''].append(wo.shape[1])
    return N_betti

def computeBettiNumbers(ordMatr, minDim=0, maxDim=2):
    N_max = binom(np.size(ordMatr,1))
    if ordMatr.ndim == 2:
        N_trial = 1
    else:
        N_trial = np.size(ordMatr,0)
    N_betti = {}
    N_betti_arr = {}
    for dim in range(minDim,maxDim+1):
        N_betti_arr['Betti'+str(dim)] = np.zeros((N_max+1,N_trial))
    for m in tqdm(range(0,N_trial), desc="Loading..."):
        N_betti[m] = computeBettiNumbers_single(ordMatr[m], minDim, maxDim)
        for dim in range(minDim,maxDim+1):
            N_betti_arr['Betti'+str(dim)][:,m] = N_betti[m]['Betti'+str(dim)]
    return N_betti_arr

# Technical

In [25]:
import math

def binom(num):
    return int(math.factorial(num)/(2*math.factorial(num-2)))

In [1]:
from scipy.stats import rankdata

def vectorOrder(values):
    order = (rankdata(values) - 1).astype(int)
    if np.std(order)==0:
        order = np.ones((np.size(order)))*np.min(values)
    return order

In [9]:
import numpy as np

def createTriMatr_upper(values, size, offDiag = 0):
    upper = np.zeros((size, size))
    upper[np.triu_indices(size, offDiag)] = values
    return(upper)

def createTriMatr_lower(values, size, offDiag = 0):
    lower = np.zeros((size, size))
    lower[np.tril_indices(size, offDiag)] = values
    return(lower)

## Plot

In [2]:
def projection(N_betti,ma = 1):
    N = np.size(N_betti['Betti0'],1)
    x = np.zeros((3,N))
    xs = np.zeros((2,1))
    ys = np.zeros((2,1))
    for i in range(N):
        x[0,i] = np.sum(N_betti['Betti0'][:,i])/N_betti['Betti0'][1,i] #slope_no_time(N_betti['Betti0'][:,i])
        x[1,i] = np.sum(N_betti['Betti1'][:,i])
        x[2,i] = i
    return x

In [3]:
def plot_points(points_current):
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x = points_current[0], 
        y = points_current[1],
        z = points_current[2],
        mode='markers', 
        showlegend=False,
        marker={
            'color': points_current[2], 
            'colorscale': 'Rainbow', 
            'cmax' : points_current[2].max(),
            'showscale': True, 
            'colorbar' : dict(thickness=10)
        }
        ))
    return fig